In [24]:
require 'countries'
require 'numo/narray'
require 'nyaplot'

false

## 注意

81でつくったdocをテキストファイルに出力したやつ(../data/enwiki_for_knock81.txt)を使う

```ruby
doc.each do |sentence|
  puts sentence
end
```

しただけ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．

In [27]:
require 'countries'
require 'numo/narray'
require '../ruby/chapter09/knock87.rb'

countries = ISO3166::Country.all.map{|obj| obj.name.gsub(/\s/, '_')}
token2id, _, mat = Marshal.load(open('../data/knock90.dump', 'rb'))

data = countries.map {|country| token2id[country] ? mat[token2id[country]] : nil }.select {|arr| arr}
names = countries.map {|country| token2id[country] ? country : nil}.select {|arr| arr}
mat = Numo::Float64[*data]


# TODO: random initialize should be done with normal distribution
tsne_mat = Numo::Float64.new(mat.shape[0], 2).rand

# perp = 0.1
num_iter = 10
eta = 0.01
# momentum = alpha

# compute p_{i, j}
p = Numo::Float64.zeros(mat.shape[0], mat.shape[0])
q = Numo::Float64.zeros(mat.shape[0], mat.shape[0])

# I compute denominator here because denominator of p_{i, j} (appeared in Equation (3)) is constant when X are given.
omega = 1 # TODO: need to compute a variance
denominator = 0
tsne_mat.shape[0].size.times do |k|
  tsne_mat.shape[0].size.times do |l|
    next if k == l
    elem = mat[k, true] - mat[l, true]
    denominator += Math.exp(- elem.transpose.dot(elem) / omega)
  end
end


tsne_mat.shape[0].times do |i|
  tsne_mat.shape[0].times do |j|
    elem1 = mat[i, true] - mat[j, true]
    numerator = Math.exp(- elem1.transpose.dot(elem1) / omega)
    p_j_i = numerator / denominator
    elem2 = mat[j, true] - mat[j, true]
    numerator = Math.exp(- elem2.transpose.dot(elem2) / omega)
    p_i_j = numerator / denominator
    p[i, j] = p_i_j + p_j_i
  end
end


num_iter.times do |t|
  denominator = 0
  tsne_mat.shape[0].size.times do |k|
    tsne_mat.shape[0].size.times do |l|
      next if k == l
      elem = tsne_mat[k, true] - tsne_mat[l, true]
      denominator += 1 + elem.transpose.dot(elem)
    end
  end

  tsne_mat.shape[0].size.times do |i|
    tsne_mat.shape[0].size.times do |j|
      # compute low-dimensional affinities q_{i, j} using Equation 4
      elem1 = tsne_mat[i, true] - tsne_mat[j, true]
      numerator = 1 + elem1.transpose.dot(elem1)

      # NOTE: numerator^{-1} / denominator^{-1}
      q[i, j] = denominator / numerator
    end
  end

  # compute gradient (using Equation 5)
  gradient = Numo::Float64.zeros(tsne_mat.shape)
  tsne_mat.shape[0].times do |i|
    # numerator = 0
    tsne_mat.shape[0].times do |j|
      elem1 = p[i, j] - q[i, j] # scalar
      elem2 = (tsne_mat[i, true] - tsne_mat[j, true]) # vector
      elem3 = 1 / (1 + elem2.transpose.dot(elem2)) # scalar

      dcdy_i = 4 * elem1 * elem2 / elem3

      # TODO: use for loop
      gradient[i, 0] = dcdy_i[0]
      gradient[i, 1] = dcdy_i[1]
    end
  end

  # update
  tsne_mat = tsne_mat + eta * gradient
end


x = tsne_mat[true, 0]
y = tsne_mat[true, 1]

plt = Nyaplot::Plot.new
plt.add(:scatter, x, y)
plt.show()

#<Nyaplot::Frame:0x0055980d297368 @properties={:panes=>[#<Nyaplot::Plot:0x0055980d3e0300 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0055980d297f70 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1"}, :data=>"74d3313b-4373-48b3-8229-4c626b985a8c"}, @xrange=[-0.48323061714373156, 1.0823662593696293], @yrange=[-0.6485023021430414, 1.0212726608290776]>], :options=>{:zoom=>true, :width=>700, :xrange=>[-0.48323061714373156, 1.0823662593696293], :yrange=>[-0.6485023021430414, 1.0212726608290776]}}>], :data=>{"74d3313b-4373-48b3-8229-4c626b985a8c"=>#<Nyaplot::DataFrame:0x0055980d3e1638 @name="74d3313b-4373-48b3-8229-4c626b985a8c", @rows=[{:data0=>0.5623281574779382, :data1=>-0.11929016440724052}, {:data0=>0.5438126557990508, :data1=>0.16778313147541576}, {:data0=>0.8748680985931107, :data1=>-0.05951382718699825}, {:data0=>-0.03922628643350887, :data1=>-0.3121512902900364}, {:data0=>0.5395014679462822, :data1=>0.9015367733647828}, {:data0=>0.8729543457273328, :data1=>0.5118118833823753}, {:data0=>0.9940005663970597, :data1=>0.7440055233292503}, {:data0=>0.8489371947512794, :data1=>0.5301880545797815}, {:data0=>1.0477914826158539, :data1=>0.14309094499099828}, {:data0=>0.38671063303490594, :data1=>0.9045838154049047}, {:data0=>0.2417903824424263, :data1=>-0.30606105933916566}, {:data0=>0.41295989294892166, :data1=>1.000211369346803}, {:data0=>0.9499807591064633, :data1=>0.979025241398045}, {:data0=>0.2863969528864627, :data1=>0.5337981910110402}, {:data0=>0.1603939220058157, :data1=>0.29300929193674513}, {:data0=>0.4607481815501188, :data1=>0.38618968832939465}, {:data0=>0.7563681172480755, :data1=>-0.17883618319925582}, {:data0=>0.6887054094733263, :data1=>-0.26131515367463515}, {:data0=>1.075382645727933, :data1=>-0.2386959519109079}, {:data0=>1.056566834290262, :data1=>0.18566523117288053}, {:data0=>-0.27579468674655583, :data1=>-0.5067407050838181}, {:data0=>-0.09006383051250832, :data1=>-0.06231193880419567}, {:data0=>0.7581588173393791, :data1=>0.5389982235387123}, {:data0=>-0.08701008018835305, :data1=>0.3028907946410154}, {:data0=>0.783399158381676, :data1=>0.9597680637211309}, {:data0=>0.7366068048352749, :data1=>0.9499537008986182}, {:data0=>0.34125331386724167, :data1=>0.4632271351497826}, {:data0=>0.8071156082761605, :data1=>0.7622133332631812}, {:data0=>0.7321132497757411, :data1=>0.7946684716519077}, {:data0=>0.04167175060787702, :data1=>0.3852951307862111}, {:data0=>0.007120525325319354, :data1=>0.6879935501694385}, {:data0=>0.9140348970055591, :data1=>0.7752717853869684}, {:data0=>0.44281003575592065, :data1=>-0.3026288756291268}, {:data0=>0.6921204455934203, :data1=>0.14115663540792972}, {:data0=>0.33394705766443816, :data1=>-0.4091342393139434}, {:data0=>0.007314119256576317, :data1=>0.973246559803136}, {:data0=>0.2521404805141036, :data1=>0.7056060037342297}, {:data0=>0.5888641477640878, :data1=>0.18741444299860674}, {:data0=>-0.022484205144686773, :data1=>0.1843198718994489}, {:data0=>0.2207544246937151, :data1=>-0.2047527776078703}, {:data0=>0.22725373670208623, :data1=>0.6503850274471509}, {:data0=>0.7188205657012906, :data1=>-0.16248063243914068}, {:data0=>0.593533956421962, :data1=>1.017747396956251}, {:data0=>0.662202412185796, :data1=>0.667644768215622}, {:data0=>-0.1932269953261102, :data1=>0.4785458934643009}, {:data0=>-0.15273960849930487, :data1=>0.3308048325887271}, {:data0=>-0.0736032263934889, :data1=>0.06566291441370965}, {:data0=>-0.14707461653499218, :data1=>0.5340127917844854}, {:data0=>0.20055934294195937, :data1=>0.6545220364753369}, {:data0=>-0.1677093944469366, :data1=>-0.4705065794788098}, {:data0=>0.3084243815729675, :data1=>0.28679234547113075}, {:data0=>0.9746664387731585, :data1=>0.3499814800480265}, {:data0=>0.7615451826972609, :data1=>0.9127566563655088}, {:data0=>0.7809915467459375, :data1=>0.9040589073421691}, {:data0=>0.008706424395100436, :data1=>-0.18059973243715957}, {:data0=>0.3237010199902518, :data1=>0.7477281225264278}, {:data0=>0.182614024155677, :data1=>1.0017